In [2]:
import pandas as pd
import plotly.express as px
from plotly.io import show
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import numpy as np

from fastf1.ergast import Ergast
import fastf1.plotting
import fastf1

In [5]:
def get_results(season):

    """wip"""

    ergast = Ergast()
    races = ergast.get_race_schedule(season)

    results = []

    # For each race in the season
    for rnd, race in races['raceName'].items():

        # Get results
        temp = ergast.get_race_results(season=season, round=rnd + 1).content[0]

        # If there is a sprint, get the results
        sprint = ergast.get_sprint_results(season=season, round=rnd + 1)

        if sprint.content and sprint.description['round'][0] == rnd + 1:

            # Merge sprint results
            temp = pd.merge(temp, sprint.content[0], on=['driverCode', 'constructorName'], how='left')

            # Add sprint points and race points to get the total
            temp['points'] = temp['points_x'] + temp['points_y']

            # Change names columns
            temp = temp.rename(columns={'grid_x': 'grid', 'position_x': 'position', 'grid_y': 'grid_sprint', 'position_y': 'position_sprint'})

            # Keep useful columns
            temp = temp[['constructorName', 'driverCode', 'points', 'grid', 'position', 'grid_sprint', 'position_sprint']]
            temp['delta_sprint'] = temp['grid_sprint'] - temp['position_sprint']

        # If there is not any sprint, we just set NaN 
        else:
            temp = temp[['constructorName', 'driverCode', 'points', 'grid', 'position']]
            temp['grid_sprint'] = np.nan
            temp['position_sprint'] = np.nan
            temp['delta_sprint'] = np.nan

        # Add round number and Grand Prix name
        temp['round'] = rnd + 1
        temp['race'] = race.removesuffix(' Grand Prix')
        temp['delta'] = temp['grid'] - temp['position']

        # Add to results
        results.append(temp)

    # Append all races into a single dataframe
    results = pd.concat(results)

    return results

In [6]:
df_2022 = get_results(2022)

In [7]:
df_2022

,constructorName,driverCode,points,grid,position,grid_sprint,position_sprint,delta_sprint,round,race,delta
0,Ferrari,LEC,26.0,1,1,NaN,NaN,NaN,1,Bahrain,0
1,Ferrari,SAI,18.0,3,2,NaN,NaN,NaN,1,Bahrain,1
2,Mercedes,HAM,15.0,5,3,NaN,NaN,NaN,1,Bahrain,2
3,Mercedes,RUS,12.0,9,4,NaN,NaN,NaN,1,Bahrain,5
4,Haas F1 Team,MAG,10.0,7,5,NaN,NaN,NaN,1,Bahrain,2
...,...,...,...,...,...,...,...,...,...,...,...
15,Haas F1 Team,MSC,0.0,12,16,NaN,NaN,NaN,22,Abu Dhabi,-4
16,Haas F1 Team,MAG,0.0,16,17,NaN,NaN,NaN,22,Abu Dhabi,-1
17,Mercedes,HAM,0.0,5,18,NaN,NaN,NaN,22,Abu Dhabi,-13
18,Williams,LAT,0.0,20,19,NaN,NaN,NaN,22,Abu Dhabi,1


In [8]:
def get_drivers_championship(df_results):

    """wip"""

    df = df_results.copy()

    df = df.pivot(index='driverCode', columns='round', values='points')
    # Here we have a 22-by-22 matrix (22 races and 22 drivers, incl. DEV and HUL)

    # Rank the drivers by their total points
    df['total_points'] = df.sum(axis=1)
    df = df.sort_values(by='total_points', ascending=False)
    df.drop(columns='total_points', inplace=True)

    # Use race name, instead of round no., as column names
    df.columns = df_results['race'].drop_duplicates()

    return df

Resultados de 2022. Pilotos

In [9]:
df_drivers_2022 = get_drivers_championship(df_2022)

In [11]:
df_drivers_2022.head()

race,Bahrain,Saudi Arabian,Australian,Emilia Romagna,Miami,Spanish,Monaco,Azerbaijan,Canadian,British,...,Hungarian,Belgian,Dutch,Italian,Singapore,Japanese,United States,Mexico City,São Paulo,Abu Dhabi
driverCode,,,,,,,,,,,,,,,,,,,,,
VER,0.0,25.0,0.0,34.0,26.0,25.0,15.0,25.0,25.0,6.0,...,25.0,26.0,26.0,25.0,6.0,25.0,25.0,25.0,13.0,25.0
LEC,26.0,19.0,26.0,15.0,18.0,0.0,12.0,0.0,10.0,12.0,...,8.0,8.0,15.0,18.0,18.0,15.0,15.0,8.0,15.0,18.0
PER,0.0,12.0,18.0,24.0,12.0,19.0,25.0,19.0,0.0,18.0,...,10.0,18.0,10.0,9.0,25.0,18.0,12.0,15.0,10.0,15.0
RUS,12.0,10.0,15.0,12.0,10.0,15.0,10.0,15.0,12.0,0.0,...,15.0,12.0,18.0,15.0,0.0,4.0,11.0,13.0,34.0,10.0
SAI,18.0,15.0,0.0,5.0,15.0,12.0,18.0,0.0,19.0,25.0,...,12.0,15.0,4.0,12.0,15.0,0.0,0.0,10.0,22.0,12.0


Resultados equipos

In [15]:
def get_constructor_championship(df_results):

    """wip"""

    df = df_results.copy()

    # Agrupar por 'round', 'race' y 'constructorName', y sumar los puntos
    df = df.groupby(['round', 'race', 'constructorName'])['points'].sum().reset_index()

    df = df.pivot(index='constructorName', columns='round', values='points')

    # Rank the teams by their total points
    df['total_points'] = df.sum(axis=1)
    df = df.sort_values(by='total_points', ascending=False)
    df.drop(columns='total_points', inplace=True)

    # Use race name, instead of round no., as column names
    df.columns = df_results['race'].drop_duplicates()

    return df


In [13]:
df_constructors_2022 = get_constructor_championship(df_2022)

In [14]:
df_constructors_2022.head()

race,Bahrain,Saudi Arabian,Australian,Emilia Romagna,Miami,Spanish,Monaco,Azerbaijan,Canadian,British,...,Hungarian,Belgian,Dutch,Italian,Singapore,Japanese,United States,Mexico City,São Paulo,Abu Dhabi
constructorName,,,,,,,,,,,,,,,,,,,,,
Red Bull,0.0,37.0,18.0,58.0,38.0,44.0,40.0,44.0,25.0,24.0,...,35.0,44.0,36.0,34.0,31.0,43.0,37.0,40.0,23.0,40.0
Ferrari,44.0,34.0,26.0,20.0,33.0,12.0,30.0,0.0,29.0,37.0,...,20.0,23.0,19.0,30.0,33.0,15.0,15.0,18.0,37.0,30.0
Mercedes,27.0,11.0,27.0,12.0,18.0,25.0,14.0,27.0,27.0,16.0,...,34.0,12.0,30.0,25.0,2.0,14.0,29.0,31.0,58.0,10.0
Alpine F1 Team,8.0,8.0,6.0,0.0,4.0,8.0,6.0,7.0,10.0,10.0,...,6.0,16.0,10.0,0.0,0.0,18.0,6.0,4.0,14.0,6.0
McLaren,0.0,6.0,18.0,22.0,0.0,4.0,9.0,6.0,0.0,8.0,...,6.0,0.0,6.0,6.0,22.0,1.0,8.0,8.0,2.0,11.0


## Standings chart

In [34]:
def plot_standings_chart(df):

    """wip: cambiar los resaltes para equipos, paleta de colores y estilos"""

    tag = df.index.name

    fig = px.imshow(
        df,
        text_auto=True,
        aspect='auto',  # Automatically adjust the aspect ratio
        color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                                [0.25, 'rgb(107, 174, 214)'],
                                [0.5,  'rgb(33,  113, 181)'],
                                [0.75, 'rgb(8,   81,  156)'],
                                [1,    'rgb(8,   48,  107)']],
        labels={'x': 'Race',
                'y': tag,
                'color': 'Points'}       # Change hover texts
    )

    # Remove axis titles
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')

    # Show all ticks, i.e. driver names
    fig.update_yaxes(tickmode='linear')

    # Show horizontal grid only
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                    showline=False,
                    tickson='boundaries')
    
    # Remove vertical grid
    fig.update_xaxes(showgrid=False, showline=False)

    # White background
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')     

    # Remove legend
    fig.update_layout(coloraxis_showscale=False)   

    # x-axis on top
    fig.update_layout(xaxis=dict(side='top'))           

    # Remove border margins
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))  

    show(fig)

In [35]:
plot_standings_chart(df_drivers_2022)

Hay que cambiar los resaltes para equipos

In [36]:
plot_standings_chart(df_constructors_2022)

---

Cumulative

In [37]:
df_cumulative_drivers_2022 = df_drivers_2022.fillna(0).cumsum(axis=1)

In [38]:
session = fastf1.get_session(2022, 'Monza', 'R')  # Ejemplo: Gran Premio de Italia, Carrera
session.load()

core           INFO 	Loading data for Italian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '44', '11', '4', '10', '45', '24', '31', '47', '77', '22', '6', '20', '3', '18', '14', '5']


Plotly

In [39]:
def plot_drivers_championship(df_cumulative, session, top=None):

    """wip"""

    gps = df_cumulative.columns
    if top:
        drivers = df_cumulative.index[:top]
    else:
        drivers = df_cumulative.index

    fig = go.Figure()

    for driver in drivers:
        points = df_cumulative.loc[driver, gps]
        
        try:
            style = fastf1.plotting.get_driver_style(driver, style=['color', 'linestyle'], session=session)
            dash = 'solid' if style['linestyle'] == 'solid' else 'dash'
            fig.add_trace(go.Scatter(
                x=gps,
                y=points,
                mode='lines+markers',
                name=driver,
                line=dict(color=style['color'], dash=dash),
                marker=dict(symbol='circle')
            ))

        except KeyError:
            print(f"Style for driver {driver} not found")


    fig.update_layout(
        title=f'Championship Evolution of Drivers - {session.event.EventDate.year} Season',
        xaxis_title='Round',
        yaxis_title='Cumulative Points',
        legend_title='Driver',
        xaxis=dict(tickmode='array', tickvals=gps),
        yaxis=dict(gridcolor='lightgray', zerolinecolor='lightgray'),
        template='plotly_dark',
        width=900,
        height=600
    )

    fig.show()


In [40]:
plot_drivers_championship(df_cumulative_drivers_2022, session, top=10)

req            INFO 	Using cached data for driver_info


## Teams

In [41]:
df_cumulative_constructors_2022 = df_constructors_2022.fillna(0).cumsum(axis=1)

In [42]:
def plot_drivers_championship(df_cumulative, session, top=None):

    """wip"""

    gps = df_cumulative.columns
    if top:
        constructors = df_cumulative.index[:top]
    else:
        constructors = df_cumulative.index

    fig = go.Figure()

    for team in constructors:
        points = df_cumulative.loc[team, gps]
        
        try:
            color = fastf1.plotting.get_team_color(team, session=session)

            fig.add_trace(go.Scatter(
                x=gps,
                y=points,
                mode='lines+markers',
                name=team,
                line=dict(color=color),
                marker=dict(symbol='circle')
            ))

        except KeyError:
            print(f"Style for constructor {team} not found")

    fig.update_layout(
        title=f'Championship Evolution of Teams - {session.event.EventDate.year} Season',
        xaxis_title='Round',
        yaxis_title='Cumulative Points',
        legend_title='Team',
        xaxis=dict(tickmode='array', tickvals=gps),
        yaxis=dict(gridcolor='lightgray', zerolinecolor='lightgray'),
        template='plotly_dark',
        width=900,
        height=600
    )

    fig.show()
    

In [43]:
plot_drivers_championship(df_cumulative_constructors_2022, session, top=None)

---

## Grid

In [44]:
# Auxiliar dataframe with championship positions

def _get_df_championship(df_results):

    """wip, se le pasa el pivotado"""

    df_championship = pd.DataFrame({
        'championshipPosition': range(1, len(df_results) + 1),
        'driverCode': df_results.index
    })

    return df_championship

### Delta

In [47]:
df_championship = _get_df_championship(df_drivers_2022)

In [57]:
def get_chart(df_results, df_championship, values='delta'):

    """
    wip

    values can be 'delta', 'grid' or 'position', including respective sprints
    """

    df = df_results.pivot(index='driverCode', columns='round', values=values)
    df = df.merge(df_championship, on='driverCode').set_index('driverCode').sort_values(by='championshipPosition').drop(columns='championshipPosition')

    # Use race name, instead of round number, as column names
    df.columns = df_results['race'].drop_duplicates()

    df.dropna(axis=1, how='all', inplace=True)

    return df


In [58]:
df = get_chart(df_2022, df_championship, values='delta_sprint')

df

race,Emilia Romagna,Austrian,São Paulo
driverCode,,,
VER,0.0,0.0,-2.0
LEC,0.0,0.0,4.0
PER,4.0,8.0,4.0
RUS,0.0,0.0,2.0
SAI,6.0,0.0,3.0
HAM,-1.0,1.0,5.0
NOR,-2.0,4.0,-3.0
OCO,3.0,-1.0,-11.0
ALO,-4.0,-12.0,-11.0
